# GraphSAGE inference

In [ ]:
from grblas import *
import grblas as gb
import numpy as np
import inspect

In [ ]:
a = np.array(
    [
        [0, 1, 0, 1, 1, 0, 0, 0],
        [1, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1],
        [1, 1, 0, 0, 1, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
    ]
)
A = gb.io.from_numpy(a)
A = ss.concat([[A, A], [A, A]])
A

In [ ]:
nnodes = A.nrows
ndims = [7, 11, 13]
nsamples = [2, 3]
assert len(ndims) == len(nsamples) + 1

In [ ]:
# Let's randomly create features and weights
features = Matrix.ss.import_fullr(np.random.rand(nnodes, ndims[0]) - 0.4)
weights = []
prev_dim = ndims[0]
for ndim in ndims[1:]:
    weights.append(Matrix.ss.import_fullr(np.random.rand(2*prev_dim, ndim) - 0.4))
    prev_dim = ndim

In [ ]:
# We could easily have a bias term.  Should we?

In [ ]:
print('A.shape:', A.shape)
print('features.shape:', features.shape)
for i, W in enumerate(weights):
    print(f'weights[{i}].shape:', W.shape)

### Algorithm 1 from https://arxiv.org/pdf/1706.02216.pdf

In [ ]:
h = features
for k in range(len(nsamples)):
    # Randomly select neighborhoods
    neighborhoods = A.ss.selectk_rowwise("random", nsamples[k])
    next_h = Matrix.new(h.dtype, nrows=nnodes, ncols=ndims[k + 1])
    for v in range(nnodes):
        # Sample neighborhood
        neighborhood = ss.diag(neighborhoods[v, :].new())
        # Aggregate neighborhood h vectors
        neighborhood_hs = op.any_second(neighborhood @ h).new()
        neighbor_aggregated = neighborhood_hs.reduce_columnwise(agg.mean).new()
        # Concat h vectors together (a Vector concat function may be handy!)
        # We could also split the weight matrices in two and either concat the results
        # or keep the results separate until the very, very end.
        size = neighbor_aggregated.size
        h_concat = Vector.new(h.dtype, size=2*size)
        h_concat[:size] = h[v, :].new()
        h_concat[size:] = neighbor_aggregated
        # Apply weight matrix
        val = op.plus_times(h_concat @ weights[k]).new()
        # Could apply bias term here
        # Apply ReLU
        val = op.max(val, 0).new()  # We could make this sparse by using select with GT_ZERO
        # Normalize (could also do this on next_h all at once)
        denom = val.reduce(agg.L2norm).new()  # L2norm is same as hypot monoid: (x**2 + y**2)**0.5
        val = binary.truediv(val, denom).new()
        # A function to stack Vectors into a Matrix may be handy! (as would dense arrays)
        next_h[v, :] = val
    h = next_h

In [ ]:
h

### Algorithm 2 from https://arxiv.org/pdf/1706.02216.pdf

In [ ]:
# The batch we're interested in (could be any number of nodes)
batch = Vector.new(A.dtype, size=nnodes)
batch[0] = True

In [ ]:
Bs = [None for _ in range(len(nsamples))]
Bs.append(batch)
neighborhoods = [None for _ in range(len(nsamples))]

In [ ]:
# Compute upfront the neighborhoods for the nodes we'll need
for k in range(len(nsamples), 0, -1):
    B = Bs[k]
    D = ss.diag(B)
    subset = op.any_second(D @ A).new()
    neighborhoods[k - 1] = subset.ss.selectk_rowwise('random', nsamples[k - 1])
    nodes = neighborhoods[k - 1].reduce_columnwise(op.any).new()
    Bs[k - 1] = op.any(B | nodes).new()

In [ ]:
h = op.any_second(ss.diag(Bs[0]) @ features).new()
for k in range(len(nsamples)):
    # Randomly select neighborhoods
    next_h = Matrix.new(h.dtype, nrows=nnodes, ncols=ndims[k + 1])
    B = Bs[k + 1]
    indices, _ = B.to_values()
    for v in indices:
        # Sample neighborhood
        neighborhood = ss.diag(neighborhoods[k][v, :].new())
        
        # The rest is the same as above in algorithm 1
        
        # Aggregate neighborhood h vectors
        neighborhood_hs = op.any_second(neighborhood @ h).new()
        neighbor_aggregated = neighborhood_hs.reduce_columnwise(agg.mean).new()
        # Concat h vectors together (a Vector concat function may be handy!)
        # We could also split the weight matrices in two and either concat the results
        # or keep the results separate until the very, very end.
        size = neighbor_aggregated.size
        h_concat = Vector.new(h.dtype, size=2*size)
        h_concat[:size] = h[v, :].new()
        h_concat[size:] = neighbor_aggregated
        # Apply weight matrix
        val = op.plus_times(h_concat @ weights[k]).new()
        # Could apply bias term here
        # Apply ReLU
        val = op.max(val, 0).new()  # We could make this sparse by using select with GT_ZERO
        # Normalize (could also do this on next_h all at once)
        denom = val.reduce(agg.L2norm).new()  # L2norm is same as hypot monoid: (x**2 + y**2)**0.5
        val = binary.truediv(val, denom).new()
        # A function to stack Vectors into a Matrix may be handy! (as would dense arrays)
        next_h[v, :] = val
    h = next_h

In [ ]:
h